## *Data Cleaning, Preparation & Visualization*<br>
#### Two of the datasets used in this process are taken from [Our World in Data](https://ourworldindata.org/)<br>
The main article in use is [Plastic Pollution](https://ourworldindata.org/plastic-pollution).
The focus of this study is to:<br>
    
   <ul> <li>Understand the total waste produced by each country in the year 2010, 
    as well as the total waste mismanaged by the countries for the same year.</li><br>
   <li>We would also see if there is any relation between waste and the GDP per capita of that entity.</li></ul><br>

### A global scale visualization of the following data has been created on [Tableau Public](https://public.tableau.com/views/PlasticWaste_16243443280510/Story1?:language=en-GB&:display_count=n&:origin=viz_share_link)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

### Following is a dataset containing plastic waste per person in Kg/day

In [ ]:
df = pd.read_csv('../input/plastic-datasets/per-capita-plastic-waste-vs-gdp-per-capita.csv')

In [ ]:
df.columns
df.rename(columns={'GDP per capita, PPP (constant 2011 international $)': 'GDP per capita in PPP', 
                   'Total population (Gapminder, HYDE & UN)': 'Total Population',
                    'Per capita plastic waste (kg/person/day)': 'Waste per person(kg/day)'}, inplace=True)

In [ ]:
# removing entities/countries with incomplete/missing data
incomplete_data_index = df[(df['Total Population'].isna()) & (df['GDP per capita in PPP'].isna())].index
df.drop(incomplete_data_index, inplace=True)

In [ ]:
# new dataframe that takes in the required data (by year 2010)
data = df[df['Year'] == 2010]
data = data.drop(columns='Continent')

In [ ]:
# retrieving continent names (from 2015 data)
con_names = df[df['Year'] == 2015]
data['Continent'] = con_names['Continent'].values

In [ ]:
# dropping rows with missing Continent values using index
miss_index = data[data['Continent'].isna()].index
data.drop(miss_index, inplace=True)

In [ ]:
# dropping rows with missing per person waste generation values
data = data[data['Waste per person(kg/day)'].notna()]

waste_gener = data.reset_index().drop('index', axis=1)

In [ ]:
waste_gener.head()

### Following is a dataset containing mismanaged plastic waste per person in Kg/day

In [ ]:
df2 = pd.read_csv('../input/plastic-datasets/per-capita-mismanaged-plastic-waste-vs-gdp-per-capita.csv')

In [ ]:
df2.rename(columns={'Per capita mismanaged plastic waste': 'Mismanaged waste per person(kg/day)',
                     'GDP per capita, PPP (constant 2011 international $)': 'GDP per capita in PPP',
                     'Total population (Gapminder, HYDE & UN)': 'Total Population'}, inplace=True)
df2.drop('Continent', axis=1, inplace=True)

In [ ]:
# new dataframe for the required data
data2 = df2[df2['Year'] == 2010]

In [ ]:
# dropping rows with missing mismanaged waste values
data2 = data2[data2['Mismanaged waste per person(kg/day)'].isna() != True]

waste_misma = data2.reset_index().drop('index', axis=1)

In [ ]:
# joining both the dataframes
plastic_waste = pd.merge(waste_gener, waste_misma, how='inner')

In [ ]:
# rearranging columns in the dataframe
plastic_waste.columns.tolist()
col_list = ['Entity','Code','Year','Waste per person(kg/day)','Mismanaged waste per person(kg/day)',
           'GDP per capita in PPP','Total Population','Continent']
plastic_waste = plastic_waste[col_list]

# rounding the values per person
plastic_waste.iloc[:, 3:5] = np.around(plastic_waste[['Waste per person(kg/day)', 
                                                      'Mismanaged waste per person(kg/day)']], decimals=2)

# changing datatype
plastic_waste['Total Population'] = plastic_waste['Total Population'].astype(int)

### Generating Total waste and Total mismanaged waste by country<br>
<ul><li>Total waste is achieved by using the product of waste generated per person per day and the total population of that country.</li><br>
<li>Total mismanaged waste is achieved by using the product of mismanaged waste per person a day and the total population of that country.</li></ul><br>

<i>Both are then multiplied by 365 to get the value for a year.</i>

In [ ]:
plastic_waste['Total waste(kgs/year)'] = ((plastic_waste['Waste per person(kg/day)'] * 
                                    plastic_waste['Total Population']) * 365)
plastic_waste['Total waste mismanaged(kgs/year)'] = ((plastic_waste['Mismanaged waste per person(kg/day)'] * 
                                    plastic_waste['Total Population']) * 365)


plastic_waste.head()

### Following graphs (Figure 1 & 2) shows the relationship between:<br>
####  1. Mismanaged waste & GDP
####  2. Waste Generated & GDP<br>

<b><i>Two Hypotheses were made</i></b>, proposing a relation between the GDP and waste generated/waste mismanaged per person a day.<br>

   <ol><li>More the GDP per Capita of an entity more would be the waste mismanaged per person.</li><br>
   <li>More the GDP per Capita of an enity more would be the waste generation per person.</li></ol>

In [ ]:
%matplotlib inline

plt.style.use('seaborn')
plt.figure(1, figsize=(12,8))

plt.scatter(plastic_waste['GDP per capita in PPP'], plastic_waste['Mismanaged waste per person(kg/day)'])
plt.title('Waste Mismanaged by GDP', loc='center', fontsize=14)
plt.ylabel('Mismanaged waste per person(kg/day)', loc='center', fontsize=12)
plt.xlabel('GDP per capita in PPP', fontsize=12)

sns.regplot(x='GDP per capita in PPP', y='Mismanaged waste per person(kg/day)', data=plastic_waste,
           scatter_kws={'color': '#34568B'}, line_kws={'color': '#650021'})

plt.show()

In [ ]:
%matplotlib inline

plt.style.use('seaborn')
plt.figure(2, figsize=(12,8))

plt.scatter(plastic_waste['GDP per capita in PPP'], plastic_waste['Waste per person(kg/day)'])
plt.title('Waste Generated by GDP', loc='center', fontsize=14)
plt.ylabel('Waste per person(kg/day)', loc='center', fontsize=12)
plt.xlabel('GDP per capita in PPP', fontsize=12)

sns.regplot(x=plastic_waste['GDP per capita in PPP'], y=plastic_waste['Waste per person(kg/day)'],
           scatter_kws={'color': '#CD212A'}, line_kws={'color': '#380282'})

plt.show()

## By the above presented graphs it can be seen that:<br>
<ul><li>Mismanaged waste <b><u>does not</u></b> increase with an increase in entity's GDP per capita.
        Hence, the evidence is enough to reject the claim.</li><br>
    <li>Generated waste <b><i><u>tend to</u></i></b> increase with entity's GDP per capita.
            Hence, there is enough evidence to support the claim.</li></ul>

In [ ]:
# creating a CSV file of the cleaned data
# if the file already exists in the folder, the file will not be created again

# plastic_waste.to_csv('datasets/plastic_waste.csv')